# 1. 这章在深度学习体系中的位置

* **模块定位**：

  * **前向传播**：线性层产生 *logits*（未归一化得分） → Softmax 把 logits 变成概率分布。
  * **损失函数**：使用**交叉熵（cross-entropy）/负对数似然（NLL）**。
  * **反向传播**：核心梯度公式是 `∂L/∂o = ŷ − y`（和 MSE 的“预测减真实”形式异曲同工）。
* **任务类型**：**多类单标签分类**（与二分类的 logistic 回归一脉相承）。
* **概率观点**：把分类看作“从离散分布中抽样”，用极大似然估计（MLE）来拟合。

---

# 2. 问题设定与标签表示（One-Hot）

* **输入**：特征向量 $\mathbf{x}\in\mathbb{R}^d$（例：2×2灰度图 → 4维）。
* **类别数**：$q$。
* **标签**：独热向量 $\mathbf{y}\in\{0,1\}^q$，且 $\sum_j y_j=1$。
* **模型（线性得分/仿射映射）**：

  $$
  \mathbf{o}=\mathbf{W}\mathbf{x}+\mathbf{b}\in\mathbb{R}^q
  $$

  这里 $\mathbf{o}$ 叫 **logits**；多输出就像一个全连接层。

---

# 3. Softmax 激活与基本性质

* **定义**：

  $$
  \hat y_i=\frac{e^{o_i}}{\sum_{j=1}^q e^{o_j}} \quad\Rightarrow\quad \hat{\mathbf{y}}=\mathrm{softmax}(\mathbf{o})
  $$
* **性质**：

  * $\hat y_i\ge 0$，$\sum_i \hat y_i=1$（合法概率分布）。
  * **保持排序**：$\arg\max_i \hat y_i=\arg\max_i o_i$（选类时不必真的计算 softmax）。
  * **温度 $T$**（了解即可）：$\mathrm{softmax}(o/T)$ 控制分布“尖/平”（$T\!\downarrow$ 更尖锐，$T\!\uparrow$ 更平滑）。

---

# 4. 小批量向量化（高效实现）

* **批输入**：$\mathbf{X}\in\mathbb{R}^{n\times d}$、$\mathbf{W}\in\mathbb{R}^{d\times q}$、$\mathbf{b}\in\mathbb{R}^{1\times q}$
* **前向**：

  $$
  \mathbf{O}=\mathbf{X}\mathbf{W}+\mathbf{b},\quad \hat{\mathbf{Y}}=\mathrm{softmax}(\mathbf{O})\ (\text{逐行})
  $$
* **复杂度**：主开销是矩阵乘 $\mathcal{O}(ndq)$。

---

# 5. 损失函数：交叉熵 / 负对数似然（来自 MLE）

* **数据似然（独立样本）**：$P(\mathbf{Y}\mid\mathbf{X})=\prod_i P(\mathbf{y}^{(i)}\mid\mathbf{x}^{(i)})$。
* **最小化负对数似然**等价于**最小化交叉熵**：

  $$
  l(\mathbf{y},\hat{\mathbf{y}})=-\sum_{j=1}^q y_j\log \hat y_j
  $$

  对 one-hot 来说就是取真实类的 $-\log \hat y_{\text{true}}$。

---

# 6. 关键梯度（推导核心）

把 softmax 代入交叉熵，可得对任意 logit $o_j$：

$$
\frac{\partial l}{\partial o_j}=\hat y_j-y_j
$$

* **含义**：梯度 = “预测概率” − “真实分布”。
* **实践价值**：实现简单稳定，也解释了为何 softmax+CE 的梯度行为像回归里的“误差”。

---

# 7. 数值稳定（log-sum-exp 技巧）

* 直接算 $e^{o}$ 容易溢出/下溢。稳定做法：

  $$
  \log\sum_k e^{o_k}
  = b + \log\sum_k e^{o_k-b},\quad b=\max_k o_k
  $$
* 框架里通常使用 `log_softmax` + NLLLoss 或 `CrossEntropyLoss`（其内部已做稳定处理）。

---

# 8. 信息论视角（帮助直觉）

* **熵**：$H[P]=\mathbb{E}_{j\sim P}[-\log P(j)]$ ——“最少需要多少 nats/bits 编码来自 $P$ 的数据”。
* **交叉熵**：$H(P,Q)=\mathbb{E}_{j\sim P}[-\log Q(j)]$ ——“用 $Q$ 来编码来自 $P$ 的数据要花多少”。
* **最优性**：当 $Q=P$ 时 $H(P,Q)$ 取最小，且等于 $H(P)$。
* **与训练目标**：最小化交叉熵 = 最大化似然。

---

# 9. 计算与工程考量

* **参数/计算成本**：全连接从 $d$ 到 $q$ 是 $\mathcal{O}(dq)$（大 $d,q$ 时代可用结构化矩阵/分解/压缩近似降低成本）。
* **实现要点**：

  * 用批处理、矩阵乘加速。
  * 用 `CrossEntropyLoss`（输入 logits，内部含 log-softmax + NLL）。
  * 类别极不平衡时可用 **class weights** 或 **focal loss**。

---

# 10. 常见易错点

1. **对 Softmax 输出再做 Sigmoid**（多此一举，且不正确）。
2. **先 Softmax 再取 log**：若自己实现，务必用 log-sum-exp 技巧或直接用 `log_softmax`。
3. **把 `CrossEntropyLoss` 的输入当成概率**：它需要 **logits** 而不是概率。
4. **数值下溢导致梯度消失**：高维/大幅度 logits 时尤其要注意稳定实现。
5. **混淆多类单标签 vs. 多标签多热（multi-label）**：后者需要对每个类别做独立 sigmoid + 二元交叉熵。

---

# 11. 一页“口袋公式”

* **前向**：$\mathbf{o}=\mathbf{W}\mathbf{x}+\mathbf{b},\quad \hat y_i=\frac{e^{o_i}}{\sum_j e^{o_j}}$
* **损失（单样本）**：$l=-\sum_j y_j\log\hat y_j$（one-hot 时 $=-\log \hat y_{\text{true}}$）
* **核心梯度**：$\frac{\partial l}{\partial o_j}=\hat y_j-y_j$
* **批量前向**：$\mathbf{O}=\mathbf{X}\mathbf{W}+\mathbf{b},\ \hat{\mathbf{Y}}=\mathrm{softmax}(\mathbf{O})$（逐行）
* **稳定 trick**：$b=\max_j o_j,\ \log\sum_j e^{o_j}=b+\log\sum_j e^{o_j-b}$

---

# 12. 思考题（由浅入深）

1. **推导题**：把 softmax 的雅可比矩阵 $\partial \hat y_i/\partial o_j$ 写出，并验证乘上 one-hot 的交叉熵梯度给出 $\hat{\mathbf{y}}-\mathbf{y}$。
2. **信息论**：证明 $H(P,Q)=H(P)+\mathrm{KL}(P\|Q)$，并据此解释“交叉熵最小化为什么会把 $Q$ 推向 $P$”。
3. **温度**：令 $\hat{\mathbf{y}}^{(T)}=\mathrm{softmax}(\mathbf{o}/T)$。讨论 $T\to 0$ 与 $T\to\infty$ 时分布形状与训练信号的变化。
4. **数值稳定**：把 $\log\sum_j e^{o_j}$ 的二阶导（Hessian）写出并解释其与方差/协方差的关系（凸性来源）。

---

# 13. 编程练习（循序渐进）

1. **从零实现**（NumPy / PyTorch 手写）：

   * 写 `linear(X @ W + b)`、`softmax`（用 log-sum-exp 稳定）、`cross_entropy`，
   * 验证梯度：对比手推 `∂L/∂O = ŷ − y` 与数值梯度（finite diff）。
2. **批量训练**（PyTorch）：

   * 用 `nn.Linear(d, q)` + `nn.CrossEntropyLoss()`（注意传 logits 给 loss）。
   * 在 MNIST 子集（仅 3 个数字）或合成数据上训练、画训练/验证曲线。
3. **对比实验**：

   * 观察：去掉稳定技巧是否导致 `nan` 或发散；
   * 调试：学习率、batch size、初始化方式对收敛的影响；
   * 类别不平衡：加上 `weight` 参数或 focal loss，比较指标（准确率/加权 F1）。

---

# 14. 一句话总结

> **Softmax 回归 = 线性打分 + Softmax 概率化 + 交叉熵（NLL）训练**；
> 梯度极其简洁：$\boxed{\nabla_{\mathbf{o}}L=\hat{\mathbf{y}}-\mathbf{y}}$；
> 工程上用批量矩阵乘与 log-sum-exp 确保**快且稳**。

